In [1]:
import pandas as pd

# CSV 파일 경로 설정
file_path = './tour_distances_matrix.csv'

# CSV 파일 로드
duration_matrix_df = pd.read_csv(file_path, index_col=0)

driving_time_matrix = duration_matrix_df.values
travel_point_name = duration_matrix_df.columns.to_list()

In [2]:
import numpy as np
import random
from deap import base, creator, tools, algorithms
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

total_day = 7

# 유전 알고리즘 설정
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# 각 관광지를 무작위 군집에 할당하는 함수
def create_individual():
    return [random.randint(0, total_day-1) for _ in range(len(driving_time_matrix) - 1)]

toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# OR-Tools를 사용하여 군집 내 관광지 순회 시간 계산
def calculate_cluster_time(cluster):
    # 숙소(인덱스 0)를 시작점과 종료점으로 추가
    cluster_with_base = [0] + cluster + [0]

    if len(cluster_with_base) <= 2:
        return 0  # 군집 내 관광지가 없으면 이동 시간은 0

    manager = pywrapcp.RoutingIndexManager(len(cluster_with_base), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    # 거리 콜백 정의
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return driving_time_matrix[cluster_with_base[from_node], cluster_with_base[to_node]]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        # 순회 시간 계산
        return solution.ObjectiveValue()
    else:
        return 0


# 평가 함수
def evalTour(individual):
    # 군집별로 관광지 인덱스를 분류
    clusters = [[] for _ in range(total_day)]
    for idx, cluster_id in enumerate(individual):
        clusters[cluster_id].append(idx + 1)  # 관광지 인덱스는 1부터 시작
    # 각 군집의 순회 시간 계산
    times = [calculate_cluster_time(cluster) for cluster in clusters]  # 숙소 포함하여 계산
    total_travel_time = np.sum(times)
    # 순회 시간의 표준 편차 반환
    if len(times) > 1:
        travel_time_std = np.std(times)
    else:
        travel_time_std = 0  # 단일 군집의 경우 표준 편차는 0

    alpha = 0.15
    fitness =  alpha * total_travel_time + (1-alpha) * travel_time_std

    return (fitness,)


toolbox.register("evaluate", evalTour)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# 유전 알고리즘 실행
population_size = 100
num_generations = 100

pop = toolbox.population(n=population_size)
hof = tools.HallOfFame(1, similar=np.array_equal)

result = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, halloffame=hof, verbose=True)

# 최적 개체 출력
best_individual = hof.items[0]
print("Best Individual = ", best_individual)
print("Fitness = ", evalTour(best_individual))

gen	nevals
0  	100   
1  	77    
2  	77    
3  	85    
4  	74    
5  	75    
6  	82    
7  	69    
8  	79    
9  	80    
10 	85    
11 	79    
12 	78    
13 	71    
14 	70    
15 	74    
16 	66    
17 	74    
18 	76    
19 	77    
20 	76    
21 	80    
22 	64    
23 	71    
24 	80    
25 	90    
26 	82    
27 	70    
28 	81    
29 	65    
30 	73    
31 	68    
32 	76    
33 	63    
34 	76    
35 	85    
36 	78    
37 	62    
38 	72    
39 	73    
40 	80    
41 	77    
42 	81    
43 	75    
44 	72    
45 	75    
46 	76    
47 	83    
48 	67    
49 	74    
50 	76    
51 	81    
52 	84    
53 	72    
54 	77    
55 	73    
56 	84    
57 	77    
58 	75    
59 	71    
60 	74    
61 	76    
62 	80    
63 	77    
64 	74    
65 	69    
66 	71    
67 	85    
68 	75    
69 	78    
70 	79    
71 	77    
72 	82    
73 	73    
74 	74    
75 	86    
76 	82    
77 	68    
78 	67    
79 	79    
80 	64    
81 	66    
82 	80    
83 	89    
84 	82    
85 	69    
86 	73    
87 	74    
88 	73    
89 	83    

In [3]:
def solve_tsp_for_cluster(cluster):
    # 숙소를 포함하여 TSP 문제 설정
    if len(cluster) == 0:
        return [0]  # 군집 내 관광지가 없으면 숙소만 반환
    cluster_with_base = [0] + cluster # 숙소를 포함
    manager = pywrapcp.RoutingIndexManager(len(cluster_with_base), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    # 거리 콜백
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return driving_time_matrix[cluster_with_base[from_node], cluster_with_base[to_node]]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # TSP 문제 해결
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

    solution = routing.SolveWithParameters(search_parameters)
    print(solution.ObjectiveValue())
    if solution:
        index = routing.Start(0)
        route = []
        while not routing.IsEnd(index):
            route.append(cluster_with_base[manager.IndexToNode(index)])
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        return route  # 숙소에서 시작하고 종료하는 경로 인덱스 반환
    else:
        return [0]  # 해결할 수 없는 경우 숙소만 반환
    
# 각 날짜별 군집화된 관광지를 식별하고 TSP 해결
for day in range(total_day):
    # 군집화된 관광지 인덱스 식별
    cluster = [i + 1 for i, cid in enumerate(best_individual) if cid == day]
    if not cluster:
        print(f"Day {day + 1}: No visits planned.")
        continue
    # 해당 군집에 대한 TSP 경로 해결
    tsp_route = solve_tsp_for_cluster(cluster)
    # 경로 출력
    route_names = [travel_point_name[idx] for idx in tsp_route]
    print(f"Day {day + 1}: " + " -> ".join(route_names))
    print()


6293
Day 1: Bukchon Hanok Village -> Myeongdong Cathedral -> Heunginjimun Gate -> Seoul Plaza -> Bukak Skyway Octagonal Pavilion -> Bukchon Hanok Village

7229
Day 2: Bukchon Hanok Village -> Lotte World Tower -> Bangi-dong Baekje Tombs -> Bukchon Hanok Village

6598
Day 3: Bukchon Hanok Village -> Changgyeonggung Palace -> Changdeokgung Palace -> Dongdaemun Design Plaza (DDP) -> Sungnyemun Gate -> Bosingak Belfry -> Deoksugung Stonewall Walkway -> The Blue House -> Bukchon Hanok Village

7206
Day 4: Bukchon Hanok Village -> Seodaemun Prison History Museum -> Namdaemun Market Tourist Information Center -> Culture Station Seoul 284 -> 63 Square -> National Assembly Building -> Gyeongbokgung Palace -> Bukchon Hanok Village

6349
Day 5: Bukchon Hanok Village -> Ikseon-dong Hanok Street -> Hwangudan Altar -> Seoullo 7017 -> Yongsan Family Park -> Namsan Seoul Tower -> Bukchon Hanok Village

7013
Day 6: Bukchon Hanok Village -> Samcheong-dong Alley -> Starfield Library -> Bukchon Hanok Vill

In [4]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# 이동 시간 매트릭스 로드
duration_df = pd.read_csv('./tour_distances_matrix.csv', index_col=0)

# 위치 정보 로드
geo_df = pd.read_csv('./geo_info.csv')
geo_df['Cluster'] = KMeans(n_clusters=total_day, random_state=42).fit(geo_df[['Latitude', 'Longitude']]).labels_

# 숙소 정보
base_location = 'Bukchon Hanok Village'

# 각 클러스터별 TSP 문제 해결 및 출력
for day in range(total_day):
    # 현재 클러스터의 관광지 선택
    cluster_locations = [base_location] + geo_df[geo_df['Cluster'] == day]['Name'].tolist()
    # 이동 시간 매트릭스 추출
    cluster_matrix = duration_df.loc[cluster_locations, cluster_locations].to_numpy()

    # TSP 문제 설정 및 해결
    manager = pywrapcp.RoutingIndexManager(len(cluster_matrix), 1, 0)
    routing = pywrapcp.RoutingModel(manager)
    transit_callback_index = routing.RegisterTransitCallback(lambda from_index, to_index: cluster_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)])
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    solution = routing.SolveWithParameters(search_parameters)
    # 결과 출력
    print(f'Day {day+1} Tour Order(with {solution.ObjectiveValue()}):')
    index = routing.Start(0)
    while not routing.IsEnd(index):
        print(f'{cluster_locations[manager.IndexToNode(index)]} -> ', end='')
        index = solution.Value(routing.NextVar(index))
    print(base_location)
    print()

Day 1 Tour Order(with 12030):
Bukchon Hanok Village -> Bukchon Hanok Village -> Changdeokgung Palace -> Myeongdong Cathedral -> Samcheong-dong Alley -> Ikseon-dong Hanok Street -> Gwanghwamun Square -> Deoksugung Palace -> Seoul Plaza -> Bosingak Belfry -> Hwangudan Altar -> Deoksugung Stonewall Walkway -> Bukak Skyway Octagonal Pavilion -> The Blue House -> Gyeongbokgung Palace -> Seodaemun Prison History Museum -> Bukchon Hanok Village

Day 2 Tour Order(with 7229):
Bukchon Hanok Village -> Lotte World Tower -> Bangi-dong Baekje Tombs -> Bukchon Hanok Village

Day 3 Tour Order(with 7423):
Bukchon Hanok Village -> 63 Square -> National Assembly Building -> Bukchon Hanok Village

Day 4 Tour Order(with 8257):
Bukchon Hanok Village -> National Museum of Korea -> Yongsan Family Park -> Banpo Bridge Night View -> National Library of Korea -> Bukchon Hanok Village

Day 5 Tour Order(with 5450):
Bukchon Hanok Village -> Starfield Library -> Bukchon Hanok Village

Day 6 Tour Order(with 5963):
B